## Synthetic Potential Failures Generator (Configurable)

This notebook generates a scalable, configurable synthetic dataset matching the schema of `customer_success.app_potential_failures` with full control over KPI code selection, task frequencies, durations, overlapping windows, and period/financial year edge cases.

Key capabilities:
- Generate ~15k records (configurable) across a two-year window starting 2025-05-25 (GTS EL start).
- Cover all KPI codes from `bronze.fms_dimkpiclassification` (or a selected subset via flags).
- Ensure for every KPI code at least one job spans the financial year boundary and crosses rail period boundaries.
- Create a realistic mix of short/medium/long/extra-long durations; include overlapping jobs to test duplicate tracking.
- Distribute tasks across all stations from `customer_success.dimStation` (filtering out depot/NULL sections).
- Join to `core_dimdate` to populate `Period`, `PeriodWeek`, and `PeriodYear`.
- All jobs default to `COMP` status. Optional history mode can simulate WAPPR → APPR → COMP.
- Write to Lakehouse first for validation, then optionally publish to SQL Server with identical schema.

How to use:
1) Configure flags below (source modes for dimensions: `jdbc`, `csv`, or `mock`).
2) Run the notebook top-to-bottom to generate and validate the dataset in Lakehouse (local path).
3) After validation, enable SQL write flags to publish to a SQL table with the same schema as `app_potential_failures`.


In [ ]:
# Configuration & Imports
from __future__ import annotations
import os
import random
import string
import uuid
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

# Optional: SQLAlchemy / pyodbc for SQL Server writes
try:
    import sqlalchemy as sa
except Exception:
    sa = None

# Optional: pyspark for Lakehouse writes in Fabric/Synapse if available
try:
    from pyspark.sql import SparkSession
    from pyspark.sql import functions as F
    from pyspark.sql import types as T
except Exception:
    SparkSession = None
    F = None
    T = None

@dataclass
class GeneratorConfig:
    # record volume
    total_records: int = 15000
    # date range
    start_date: datetime = datetime(2025, 5, 25)
    end_date: datetime = datetime(2027, 5, 24)
    # KPI code controls
    include_all_kpi_codes: bool = True
    selected_kpi_codes: Optional[List[str]] = None
    # frequency per KPI code multiplier (1.0 baseline)
    kpi_frequency_overrides: Optional[Dict[str, float]] = None
    # overlapping windows control
    enable_overlap_groups: bool = True
    overlap_group_fraction: float = 0.10  # 10% have close overlaps per station/date
    overlap_hours_window: int = 4
    # duration mix per KPI
    short_hours: Tuple[int, int] = (1, 4)
    medium_hours: Tuple[int, int] = (6, 24)
    long_hours: Tuple[int, int] = (24, 96)
    extra_long_hours: Tuple[int, int] = (96, 240)  # up to 10 days
    duration_mix_weights: Tuple[float, float, float, float] = (0.35, 0.35, 0.20, 0.10)
    # ensure one FY-spanning job per KPI code
    ensure_fy_spanners: bool = True
    financial_year_end_month: int = 3  # March
    financial_year_end_day: int = 31
    # rail period join
    require_period_join: bool = True
    # dimensions sources: 'jdbc' | 'csv' | 'mock'
    kpi_source_mode: str = 'mock'
    stations_source_mode: str = 'mock'
    dimdate_source_mode: str = 'mock'
    # IO paths
    local_output_dir: str = './lh_out'
    local_dim_cache_dir: str = './lh_dims'
    lakehouse_output_table: Optional[str] = None  # e.g., 'Tables/synthetic_app_potential_failures'
    # SQL Server
    enable_sql_write: bool = False
    sql_connection_string: Optional[str] = None  # e.g., 'mssql+pyodbc://...'
    sql_target_table: Optional[str] = None  # schema-qualified
    # history mode
    enable_history_mode: bool = False
    history_fraction: float = 0.15
    # allow generating non-KPI codes optionally
    enable_non_kpi: bool = False
    non_kpi_fraction: float = 0.05
    # station filters
    include_depots: bool = False

config = GeneratorConfig()

os.makedirs(config.local_output_dir, exist_ok=True)
os.makedirs(config.local_dim_cache_dir, exist_ok=True)

random.seed(42)
np.random.seed(42)


In [ ]:
# Helpers

def random_string(n: int = 8) -> str:
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=n))


def random_email(name: str) -> str:
    domain = random.choice(["example.com", "railops.org", "cleanit.io", "facilities.net"])
    return f"{name.lower()}@{domain}"


def choose_weighted(weights: Tuple[float, ...]) -> int:
    return int(np.random.choice(len(weights), p=np.array(weights)/np.sum(weights)))


def make_datetime_range(start: datetime, end: datetime) -> datetime:
    if start > end:
        start, end = end, start
    span_seconds = int((end - start).total_seconds())
    offset = random.randint(0, max(0, span_seconds - 1))
    return start + timedelta(seconds=offset)


def ensure_end_after(start: datetime, min_delta_hours: int, max_delta_hours: int) -> datetime:
    hours = random.randint(min_delta_hours, max_delta_hours)
    # randomize minutes within the hour window
    minutes = random.randint(0, 59)
    return start + timedelta(hours=hours, minutes=minutes)


def compute_financial_year(date_val: datetime, fy_end_month: int, fy_end_day: int) -> int:
    fy_end = datetime(date_val.year, fy_end_month, fy_end_day)
    return date_val.year if date_val <= fy_end else date_val.year + 1


In [ ]:
# Mock/load dimensions: KPI codes, Stations, core_dimdate

# In real mode (jdbc/csv), implement readers. For now, provide mock fallbacks.

def load_kpi_codes(mode: str, selected: Optional[List[str]] = None) -> pd.DataFrame:
    if mode == 'mock':
        # Representative KPI codes including graffiti, track cleaning, and a few others
        codes = [
            ("KPI_GRAFFITI", "Graffiti removal", "Cleanliness"),
            ("KPI_TRACKCLEAN", "Trackside cleaning", "Cleanliness"),
            ("KPI_LITTER", "Litter removal", "Cleanliness"),
            ("KPI_LIGHT", "Lighting fault", "Asset"),
            ("KPI_ESCALATOR", "Escalator fault", "Asset"),
            ("KPI_SIGNAGE", "Signage damage", "Asset"),
            ("KPI_VANDAL", "Vandalism repair", "Security"),
            ("KPI_WATER", "Water leak", "Asset"),
            ("KPI_ELECTRICAL", "Electrical fault", "Asset"),
            ("KPI_TRACKOBST", "Track obstruction", "Operations"),
        ]
        df = pd.DataFrame(codes, columns=["Instruction_Code", "KPIDescription", "KPICategory"])
    elif mode == 'csv':
        df = pd.read_csv(os.path.join(config.local_dim_cache_dir, 'bronze.fms_dimkpiclassification.csv'))
    else:  # jdbc placeholder
        # Implement your JDBC reader here if running in Fabric/Spark environment
        raise NotImplementedError("JDBC mode not implemented in this environment")

    if selected is not None and len(selected) > 0:
        df = df[df['Instruction_Code'].isin(selected)].reset_index(drop=True)
    return df


def load_stations(mode: str, include_depots: bool = False) -> pd.DataFrame:
    if mode == 'mock':
        stations = [
            ("STN001", "Central", "Main Line", "Section A"),
            ("STN002", "North", "North Line", "Section B"),
            ("STN003", "East", "East Line", "Section C"),
            ("STN004", "West", "West Line", None),  # depot-like, should be dropped if include_depots False
            ("STN005", "South", "South Line", "Section D"),
        ]
        df = pd.DataFrame(stations, columns=["StationCode", "StationName", "Line", "StationSection"])
    elif mode == 'csv':
        df = pd.read_csv(os.path.join(config.local_dim_cache_dir, 'customer_success.dimStation.csv'))
    else:
        raise NotImplementedError("JDBC mode not implemented in this environment")

    if not include_depots:
        df = df[df['StationSection'].notna()].reset_index(drop=True)
    return df


def load_dimdate(mode: str, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    if mode == 'mock':
        # Build a simple core_dimdate with rail periods (4-5 weeks cycles mocked)
        dates = pd.date_range(start=start_date - timedelta(days=120), end=end_date + timedelta(days=120), freq='D')
        period_years = []
        periods = []
        period_weeks = []
        week_in_period = 0
        period = 1
        period_year = start_date.year
        # naive rolling period model: 13 periods per year, 4 or 5 weeks alternation
        pattern = [4,4,5]*4 + [4]
        pat_idx = 0
        day_idx = 0
        for d in dates:
            if day_idx % 7 == 0:  # new week
                week_in_period += 1
                if week_in_period > pattern[pat_idx % len(pattern)]:
                    week_in_period = 1
                    period += 1
                    if period > 13:
                        period = 1
                        period_year += 1
                    pat_idx += 1
            period_weeks.append(week_in_period)
            periods.append(period)
            period_years.append(period_year)
            day_idx += 1
        df = pd.DataFrame({
            'Date': dates,
            'PeriodWeek': period_weeks,
            'Period': periods,
            'PeriodYear': period_years,
        })
    elif mode == 'csv':
        df = pd.read_csv(os.path.join(config.local_dim_cache_dir, 'core_dimdate.csv'))
        df['Date'] = pd.to_datetime(df['Date'])
    else:
        raise NotImplementedError("JDBC mode not implemented in this environment")

    return df


In [ ]:
# Generator

SCHEMA_COLUMNS = [
    ("TaskId", "nvarchar"),
    ("RecordID", "nvarchar"),
    ("Instruction_Code", "nvarchar"),
    ("Building", "nvarchar"),
    ("BuildingName", "nvarchar"),
    ("LocationName", "nvarchar"),
    ("ShortDescription", "nvarchar"),
    ("LongDescription", "nvarchar"),
    ("Reporter", "nvarchar"),
    ("ReporterEmail", "nvarchar"),
    ("Notes", "nvarchar"),
    ("ReportedDate", "datetime2"),
    ("DueBy", "datetime2"),
    ("ScheduledFor", "datetime2"),
    ("Finished", "datetime2"),
    ("Status", "nvarchar"),
    ("LoggedBy", "nvarchar"),
    ("LoggedOn", "datetime2"),
    ("ModifiedOn", "datetime2"),
    ("SLAStatus", "nvarchar"),
    ("CreatedTimestamp", "datetime2"),
    ("LastUploaded", "datetime2"),
    ("IsCurrent", "bit"),
    ("Period", "nvarchar"),
    ("PeriodWeek", "bigint"),
    ("PeriodYear", "bigint"),
    ("StationSection", "nvarchar"),
    ("KPIDescription", "nvarchar"),
    ("KPICategory", "nvarchar"),
]


def pick_duration_hours(mix: Tuple[float, float, float, float]) -> Tuple[int, int]:
    bucket = choose_weighted(mix)
    if bucket == 0:
        return config.short_hours
    if bucket == 1:
        return config.medium_hours
    if bucket == 2:
        return config.long_hours
    return config.extra_long_hours


def generate_for_kpi(
    kpi_row: pd.Series, stations: pd.DataFrame, dimdate: pd.DataFrame, num_records: int
) -> pd.DataFrame:
    records = []

    kpi_code = kpi_row['Instruction_Code']
    kpi_desc = kpi_row.get('KPIDescription', '')
    kpi_cat = kpi_row.get('KPICategory', '')

    # Preselect a proportion of overlapping jobs by station and approximate logged time
    overlap_targets = set()
    if config.enable_overlap_groups and num_records > 10:
        desired_overlap = int(num_records * config.overlap_group_fraction)
        for _ in range(desired_overlap):
            stn = stations.sample(1).iloc[0]
            base_time = make_datetime_range(config.start_date, config.end_date)
            overlap_targets.add((stn['StationCode'], base_time.replace(minute=0, second=0, microsecond=0)))

    # Ensure a FY-spanner for this KPI code if requested
    fy_spanner_created = False
    if config.ensure_fy_spanners:
        # Choose a date close to FY end and span across
        fy_end_this = datetime(config.start_date.year, config.financial_year_end_month, config.financial_year_end_day)
        start_spanner = fy_end_this - timedelta(days=random.randint(1, 10))
        end_spanner = fy_end_this + timedelta(days=random.randint(1, 15))
        # Ensure falls within global window
        start_spanner = max(config.start_date, start_spanner)
        end_spanner = min(config.end_date, end_spanner)
        stn = stations.sample(1).iloc[0]
        records.append(
            build_record(kpi_code, kpi_desc, kpi_cat, stn, start_spanner, end_spanner, status="COMP", fy_spanner=True)
        )
        fy_spanner_created = True

    for i in range(num_records - (1 if fy_spanner_created else 0)):
        stn = stations.sample(1).iloc[0]

        # LoggedOn as base, within the two-year window
        logged_on = make_datetime_range(config.start_date, config.end_date)

        # Overlap adjustment: if this station + hour bucket is tagged, cluster within overlap window
        key = (stn['StationCode'], logged_on.replace(minute=0, second=0, microsecond=0))
        if key in overlap_targets:
            # shift within +/- overlap_hours_window
            shift_hours = random.randint(-config.overlap_hours_window, config.overlap_hours_window)
            logged_on = logged_on + timedelta(hours=shift_hours)

        # Determine duration bucket and compute end
        dmin, dmax = pick_duration_hours(config.duration_mix_weights)
        finished = ensure_end_after(logged_on, dmin, dmax)

        # DueBy and ScheduledFor around LoggedOn
        due_by = logged_on + timedelta(hours=random.randint(4, 72))
        scheduled_for = logged_on + timedelta(hours=random.randint(0, 24))

        records.append(
            build_record(kpi_code, kpi_desc, kpi_cat, stn, logged_on, finished, status="COMP",
                         due_by=due_by, scheduled_for=scheduled_for)
        )

    df = pd.DataFrame(records)

    # Join to core_dimdate to add period fields based on LoggedOn date
    if config.require_period_join and not df.empty:
        dd = dimdate[["Date", "Period", "PeriodWeek", "PeriodYear"]].copy()
        dd['Date'] = pd.to_datetime(dd['Date']).dt.date
        df['LoggedOnDate'] = pd.to_datetime(df['LoggedOn']).dt.date
        df = df.merge(dd, left_on='LoggedOnDate', right_on='Date', how='left')
        df.drop(columns=['Date', 'LoggedOnDate'], inplace=True)
        # Ensure correct dtypes/names
        df['Period'] = df['Period'].astype('Int64').astype(str)
        df['PeriodWeek'] = df['PeriodWeek'].astype('Int64')
        df['PeriodYear'] = df['PeriodYear'].astype('Int64')

    return df


def build_record(
    kpi_code: str,
    kpi_desc: str,
    kpi_cat: str,
    stn: pd.Series,
    logged_on: datetime,
    finished: datetime,
    status: str = "COMP",
    due_by: Optional[datetime] = None,
    scheduled_for: Optional[datetime] = None,
    fy_spanner: bool = False,
) -> Dict[str, object]:
    short_desc = f"{kpi_desc or kpi_code} at {stn['StationName']}"
    long_desc = f"{short_desc} — {random_string(12)}"
    reporter_name = random.choice(["Alex", "Sam", "Jordan", "Casey", "Taylor", "Morgan"]) + " " + random.choice(["Lee", "Smith", "Patel", "Khan", "Brown", "Lopez"]) 
    reporter_email = random_email(reporter_name.replace(" ", "."))

    # Model SLA status roughly
    sla_status = random.choice(["Within", "Breached", "At Risk"]) if not fy_spanner else "Within"

    row = {
        "TaskId": str(uuid.uuid4()),
        "RecordID": str(uuid.uuid4()),
        "Instruction_Code": kpi_code,
        "Building": stn['Line'],
        "BuildingName": stn['StationName'],
        "LocationName": stn['StationCode'],
        "ShortDescription": short_desc,
        "LongDescription": long_desc,
        "Reporter": reporter_name,
        "ReporterEmail": reporter_email,
        "Notes": random.choice(["", "Follow-up required", "High priority", "Photo attached", "Repeat issue"]),
        "ReportedDate": logged_on,
        "DueBy": due_by or (logged_on + timedelta(hours=24)),
        "ScheduledFor": scheduled_for or (logged_on + timedelta(hours=8)),
        "Finished": finished,
        "Status": status,
        "LoggedBy": random.choice(["FMS", "CRM", "MobileApp", "Email"]),
        "LoggedOn": logged_on,
        "ModifiedOn": finished,
        "SLAStatus": sla_status,
        "CreatedTimestamp": logged_on,
        "LastUploaded": finished,
        "IsCurrent": True,
        "StationSection": stn['StationSection'],
        "KPIDescription": kpi_desc,
        "KPICategory": kpi_cat,
    }

    return row


In [ ]:
# Main orchestration

kpis = load_kpi_codes(config.kpi_source_mode, config.selected_kpi_codes)
stations = load_stations(config.stations_source_mode, include_depots=config.include_depots)
dimdate = load_dimdate(config.dimdate_source_mode, config.start_date, config.end_date)

if config.include_all_kpi_codes:
    active_kpis = kpis
else:
    if config.selected_kpi_codes:
        active_kpis = kpis[kpis['Instruction_Code'].isin(config.selected_kpi_codes)]
    else:
        active_kpis = kpis.head(1)

# Determine per-KPI record counts using frequency overrides
base_per_kpi = max(1, int(config.total_records / max(1, len(active_kpis))))
allocations = {}
remaining = config.total_records
for code in active_kpis['Instruction_Code']:
    mult = (config.kpi_frequency_overrides or {}).get(code, 1.0)
    count = max(1, int(base_per_kpi * mult))
    allocations[code] = count
    remaining -= count
# Distribute any remainder
codes_cycle = list(active_kpis['Instruction_Code'])
idx = 0
while remaining > 0 and codes_cycle:
    allocations[codes_cycle[idx % len(codes_cycle)]] += 1
    idx += 1
    remaining -= 1

frames = []
for _, krow in active_kpis.iterrows():
    count = allocations[krow['Instruction_Code']]
    frames.append(generate_for_kpi(krow, stations, dimdate, count))

result_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=[c for c,_ in SCHEMA_COLUMNS])

# Ensure schema columns exist and order
for col, _ in SCHEMA_COLUMNS:
    if col not in result_df.columns:
        result_df[col] = np.nan
result_df = result_df[[c for c,_ in SCHEMA_COLUMNS]]

print("Generated records:", len(result_df))
result_df.head(3)

In [ ]:
# Validation & summary

assert (result_df['Status'] == 'COMP').all(), "All jobs should be COMP by default"
assert result_df['Instruction_Code'].notna().all(), "All rows should have KPI codes"

by_kpi = result_df.groupby('Instruction_Code').size().sort_values(ascending=False)
print("Counts by KPI code:\n", by_kpi)

# Check rail period coverage
period_counts = result_df.groupby(['PeriodYear', 'Period']).size()
print("\nCounts by PeriodYear/Period (sample):\n", period_counts.head(20))

# Check durations histogram (Finished - LoggedOn)
_durations_hours = (
    (pd.to_datetime(result_df['Finished']) - pd.to_datetime(result_df['LoggedOn']))
    .dt.total_seconds() / 3600.0
)
print("\nDuration hours (summary):\n", _durations_hours.describe())

In [ ]:
# Write to Lakehouse (local filesystem as stand-in) and optionally to SQL Server

# Lakehouse/local: write parquet and csv
parquet_path = os.path.join(config.local_output_dir, 'synthetic_app_potential_failures.parquet')
csv_path = os.path.join(config.local_output_dir, 'synthetic_app_potential_failures.csv')

result_df.to_parquet(parquet_path, index=False)
result_df.to_csv(csv_path, index=False)
print(f"Wrote {len(result_df)} rows to {parquet_path} and {csv_path}")

# Optional Spark write (if Spark available and lakehouse_output_table provided)
if SparkSession is not None and config.lakehouse_output_table:
    spark = SparkSession.builder.getOrCreate()
    sdf = spark.createDataFrame(result_df)
    sdf.write.mode('overwrite').saveAsTable(config.lakehouse_output_table)
    print(f"Saved Spark table: {config.lakehouse_output_table}")

# Optional SQL Server write
if config.enable_sql_write:
    assert sa is not None, "sqlalchemy required for SQL write"
    assert config.sql_connection_string and config.sql_target_table, "Provide SQL connection and target table"
    engine = sa.create_engine(config.sql_connection_string, fast_executemany=True)
    with engine.begin() as conn:
        result_df.to_sql(name=config.sql_target_table.split('.')[-1],
                         con=conn,
                         schema=config.sql_target_table.split('.')[0] if '.' in config.sql_target_table else None,
                         if_exists='replace',
                         index=False,
                         dtype=None)
    print(f"Wrote to SQL Server table {config.sql_target_table}")
